# A Non-Interactive Coinjoin - BX Example

<br>
<img src="images/transaction_singleany.jpg" alt="drawing" style="" width="800px"/>



<hr style="border: 0.5px dashed #000;">

## 1. Setup wallet and test funds.

<br>

### Recreate keys which control spendable funds: `m/44'/1'/1'/0/1 ... m/44'/1'/1'/0/6`

In [ ]:
my_mnemonic=""
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)

# We will send funds from keys in account 1.

# Account 1 receiving parent: m/44'/1'/1'/0.
hd_m_44h_1h_1h_0=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0)

# Derive first two keys from account 1 with spendable funds (P2PKH outputs).

# Alice sends from this: m/44'/1'/1'/0/1
hd_m_44h_1h_0h_0_1=$(bx hd-private --index 1 $hd_m_44h_1h_1h_0)
privatekey_44h_1h_1h_0_1=$(bx hd-to-ec $hd_m_44h_1h_0h_0_1)
publickey_44h_1h_1h_0_1=$(bx hd-to-public $hd_m_44h_1h_0h_0_1 | bx hd-to-ec)

# Bob sends from this: m/44'/1'/1'/0/2
hd_m_44h_1h_0h_0_2=$(bx hd-private --index 2 $hd_m_44h_1h_1h_0)
privatekey_44h_1h_1h_0_2=$(bx hd-to-ec $hd_m_44h_1h_0h_0_2)
publickey_44h_1h_1h_0_2=$(bx hd-to-public $hd_m_44h_1h_0h_0_2 | bx hd-to-ec)


### Note the UTXO's we will be spending from.


In [ ]:
# All spendable outputs are in same tx.
previous_txid=d4e68f7e163de4c31c6d4d75a38daa015a968371c1a94e2a182e29002235a168

# Alice's UTXO:
previous_output_index_alice=0
previous_output_amount_alice=1964541

# Bob's UTXO:
previous_output_index_bob=1
previous_output_amount_bob=1964541

# For later fee calcuation:
total_previous_output_amount=$(expr $previous_output_amount_alice + $previous_output_amount_bob)

### Derive destination keys to send funds to.

In [ ]:
# We will send funds to keys in account 2.

# Account 2 receiving parent: m/44'/1'/2'/0.
hd_m_44h_1h_2h_0=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 2  \
| bx hd-private --index 0)

# Alice sends to this.
hd_m_44h_1h_2h_0_0=$(bx hd-private --index 0 $hd_m_44h_1h_2h_0)
publickeyhash_44h_1h_2h_0_0=$(bx hd-to-public $hd_m_44h_1h_2h_0_0 | bx hd-to-ec | bx sha256 | bx ripemd160)

# Bob sends to this.
hd_m_44h_1h_2h_0_1=$(bx hd-private --index 1 $hd_m_44h_1h_2h_0)
publickeyhash_44h_1h_2h_0_1=$(bx hd-to-public $hd_m_44h_1h_2h_0_1 | bx hd-to-ec | bx sha256 | bx ripemd160)


## 2. Alice signs with ANYONECANPAY | SINGLE

**Alice creates p2pkh output script.**

In [ ]:
output_script_alice=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_2h_0_0"] EQUALVERIFY CHECKSIG")


**Alice computes output amounts and fee.**

* Alice and Bob will send to outputs of equal amounts.
* Alice will compute single output amount, taking fee into consideration.

In [ ]:
# tx bytes: 4 + 1 + 2*(32+4+110+4) + 1 + 2*(8+33) + 4 = 392

tx_byte_count=$(expr 4 + 1 + 2 \* 150 + 1 + 2 \* 41 + 4)
total_output_amount=$(expr $total_previous_output_amount - 6 \* 400 - $tx_byte_count \* 1)
single_output_amount=$(expr $total_output_amount / 2)


**Alice creates transaction template for signing.**

In [ ]:
# Transaction for Alice to sign.

tx_alice=$(bx tx-encode \
--input $previous_txid:$previous_output_index_alice:4294967295 \
--output $output_script_alice:$single_output_amount) 


**Alice signs transaction.**

In [ ]:
previous_decoded_output_script_alice=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[$previous_output_index_alice].script")


In [ ]:
endorsement_alice=$(bx input-sign --index 0 --anyone --sign_type single \
$privatekey_44h_1h_1h_0_1 \
"$previous_decoded_output_script_alice" \
$tx_alice)


In [ ]:
tx_alice=$(bx input-set --index 0 \
"[$endorsement_alice] [$publickey_44h_1h_1h_0_1]" \
$tx_alice)


## 3. Bob signs with ANYONECANPAY | SINGLE

**Bob creates p2pkh output script.**

In [ ]:
output_script_bob=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_2h_0_1"] EQUALVERIFY CHECKSIG")


**Bob creates transaction template, with placeholders at index 0.**

In [ ]:
# Transaction template must include placeholder input/output,
# ...since Bob's later endorsement commits to index 1.

tx_bob=$(bx tx-encode \
--input 0000000000000000000000000000000000000000000000000000000000000000:0:4294967295 \
--input $previous_txid:$previous_output_index_bob:4294967295 \
--output 00:0 \
--output $output_script_bob:$single_output_amount) 


**Bob signs his transaction.**

In [ ]:
previous_decoded_output_script_bob=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[$previous_output_index_bob].script")


In [ ]:
# Sign input at index 1.
endorsement_bob=$(bx input-sign --index 1 --anyone --sign_type single \
$privatekey_44h_1h_1h_0_2 \
"$previous_decoded_output_script_bob" \
$tx_bob)


In [ ]:
# Input 0: Input script is left empty.
# Input 1: Set Bob's input script.
tx_bob=$(bx input-set --index 1 "[$endorsement_bob] [$publickey_44h_1h_1h_0_2]" $tx_bob)


## 3. Charlie combines Alice's and Bob's transaction data.

**Charlie extracts transaction elements from Alice.**

In [ ]:
tx_alice=$(bx tx-decode --format json $tx_alice)

tx_alice_version=$(echo $tx_alice | jq -r ".transaction.version")
tx_alice_inputs=$(echo $tx_alice | jq ".transaction.inputs")
tx_alice_input0=$(echo $tx_alice_inputs | jq ".[0]")
tx_alice_input0_prev_hash=$(echo $tx_alice_input0 | jq -r ".previous_output.hash")
tx_alice_input0_prev_index=$(echo $tx_alice_input0 | jq -r ".previous_output.index")
tx_alice_input0_script=$(echo $tx_alice_input0 | jq -r ".script")
tx_alice_input0_sequence=$(echo $tx_alice_input0 | jq -r ".sequence")
tx_alice_outputs=$(echo $tx_alice | jq ".transaction.outputs")
tx_alice_output0=$(echo $tx_alice_outputs | jq -r ".[0]")
tx_alice_output0_script=$(echo $tx_alice_output0 | jq -r ".script" | bx script-encode)
tx_alice_output0_value=$(echo $tx_alice_output0 | jq -r ".value")
tx_alice_locktime=$(echo $tx_alice | jq -r ".transaction.lock_time")


**Charlie extracts transaction elements from Bob.**

In [ ]:
tx_bob=$(bx tx-decode --format json $tx_bob)

tx_bob_version=$(echo $tx_bob | jq -r ".transaction.version")
tx_bob_inputs=$(echo $tx_bob | jq ".transaction.inputs")
tx_bob_input1=$(echo $tx_bob_inputs | jq ".[1]")
tx_bob_input1_prev_hash=$(echo $tx_bob_input1 | jq -r ".previous_output.hash")
tx_bob_input1_prev_index=$(echo $tx_bob_input1 | jq -r ".previous_output.index")
tx_bob_input1_script=$(echo $tx_bob_input1 | jq -r ".script")
tx_bob_input1_sequence=$(echo $tx_bob_input1 | jq -r ".sequence")
tx_bob_outputs=$(echo $tx_bob | jq ".transaction.outputs")
tx_bob_output1=$(echo $tx_bob_outputs | jq -r ".[1]")
tx_bob_output1_script=$(echo $tx_bob_output1 | jq -r ".script" | bx script-encode)
tx_bob_output1_value=$(echo $tx_bob_output1 | jq -r ".value")
tx_bob_locktime=$(echo $tx_bob | jq -r ".transaction.lock_time")


**Charlie creates a combined transaction.**

* Input and outputs remain at the same index (committed to by Alice's and Bob's endorsements)

In [ ]:
# Assume:
# tx_a.version = tx_b.version
# tx_a.locktime = tx_b.locktime

tx_joined=$(bx tx-encode \
--version $tx_alice_version \
--input $tx_alice_input0_prev_hash:$tx_alice_input0_prev_index:$tx_alice_input0_sequence \
--input $tx_bob_input1_prev_hash:$tx_bob_input1_prev_index:$tx_bob_input1_sequence \
--output $tx_alice_output0_script:$tx_alice_output0_value \
--output $tx_bob_output1_script:$tx_bob_output1_value \
--lock_time $tx_alice_locktime)


In [ ]:
# Set inputs into combined transaction.
tx_joined=$(bx input-set "$tx_alice_input0_script" $tx_joined)
tx_joined=$(bx input-set --index 1 "$tx_bob_input1_script" $tx_joined)


In [ ]:
bx validate-tx $tx_joined
echo $tx_joined